In [85]:
import os
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow import keras
import numpy as np
from PIL import Image

In [87]:
BATCH_SIZE = 16 # adjust the batch size
IMAGE_HEIGHT = 512  # adjust the number
IMAGE_WIDTH = 512 # adjust the number
IMG_SIZE = (IMAGE_HEIGHT, IMAGE_WIDTH)
SEED = 42

In [96]:
img_path = 'Path to image directory'
msk_path = 'Path to mask directory'
output_dir = 'Path to saving the tested images directory'
lowest_iou_dir = 'path to save the lowest IoU results'

In [98]:
smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

In [100]:
import tensorflow as tf
model = tf.keras.models.load_model("weight_file.h5", custom_objects={"dice_coef": dice_coef})

In [102]:
def create_segmentation_generator_test(img_path, batch_size):
    data_gen_args = dict(rescale=1./255)
    datagen = ImageDataGenerator(**data_gen_args)
    
    img_generator = datagen.flow_from_directory(
        img_path,
        target_size=IMG_SIZE,
        class_mode=None,
        color_mode='grayscale',
        batch_size=batch_size,
        seed=SEED,
        shuffle=False
    )
    msk_generator = datagen.flow_from_directory(
        msk_path,
        target_size=IMG_SIZE,
        class_mode=None,
        color_mode='grayscale',
        batch_size=BATCH_SIZE,
        seed=SEED,
        shuffle = False
    )
    filenames = img_generator.filenames
    return img_generator,msk_generator, filenames

## Starting from here batch size does not matter

In [104]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, f1_score

def dice_coefficient(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    denominator = np.sum(y_true) + np.sum(y_pred)
    if denominator == 0:
        return 1.0 if np.sum(y_true) == 0 and np.sum(y_pred) == 0 else 0.0
    
    return 2 * intersection / denominator

def calculate_accuracy(y_true, y_pred):
    return np.mean(y_true.flatten() == y_pred.flatten())

def binary_iou(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    union = np.sum(y_true) + np.sum(y_pred) - intersection
    if union == 0:
        return 1.0 if np.sum(y_true) == 0 and np.sum(y_pred) == 0 else 0.0
    
    return intersection / union

In [90]:
def predict_and_save_masks(model, img_datagen, msk_datagen, filenames, output_dir, lowest_iou_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    if not os.path.exists(lowest_iou_dir):
        os.makedirs(lowest_iou_dir)

    num_images = len(filenames)
    num_batches = num_images // img_datagen.batch_size
    remainder = num_images % img_datagen.batch_size

    total_accuracy = 0
    total_dice = 0
    total_iou = 0
    highest_iou = -1 
    lowest_iou = 1  
    num_images_processed = 0
    lowest_iou_img_path = None  
    lowest_iou_gt_path = None  
    lowest_iou_value = 1 

    for i in range(num_batches):
        img_batch = next(img_datagen)
        msk_batch = next(msk_datagen)
        
        pred_masks = model.predict(img_batch, verbose=0)
        
        for j, pred_mask in enumerate(pred_masks):
            original_filename = os.path.basename(filenames[i * img_datagen.batch_size + j])
            save_path = os.path.join(output_dir, f'{os.path.splitext(original_filename)[0]}.png')
            
            pred_mask_bin = (pred_mask.squeeze() > 0.5).astype(np.uint8)
            pred_mask_img = Image.fromarray(pred_mask_bin * 255)
            pred_mask_img.save(save_path)

            ground_truth_mask = msk_batch[j].squeeze().astype(np.uint8)

            if np.sum(ground_truth_mask) == 0 and np.sum(pred_mask_bin) == 0:
                continue 

            accuracy = calculate_accuracy(ground_truth_mask, pred_mask_bin)
            dice = dice_coefficient(ground_truth_mask, pred_mask_bin)
            iou = binary_iou(ground_truth_mask, pred_mask_bin)

            total_accuracy += accuracy
            total_dice += dice
            total_iou += iou
            num_images_processed += 1

            if iou > highest_iou:
                highest_iou = iou
            if iou < lowest_iou:
                lowest_iou = iou
                lowest_iou_img_path = save_path 
                lowest_iou_gt_path = os.path.join(lowest_iou_dir, f'{os.path.splitext(original_filename)[0]}_gt.png')
                gt_mask_img = Image.fromarray(ground_truth_mask * 255)
                gt_mask_img.save(lowest_iou_gt_path)

    if remainder > 0:
        img_batch = next(img_datagen)
        msk_batch = next(msk_datagen)
        pred_masks = model.predict(img_batch[:remainder], verbose=0)
        
        for j, pred_mask in enumerate(pred_masks):
            original_filename = os.path.basename(filenames[num_batches * img_datagen.batch_size + j])
            save_path = os.path.join(output_dir, f'{os.path.splitext(original_filename)[0]}.png')

            pred_mask_bin = (pred_mask.squeeze() > 0.5).astype(np.uint8)
            pred_mask_img = Image.fromarray(pred_mask_bin * 255)
            pred_mask_img.save(save_path)

            ground_truth_mask = msk_batch[j].squeeze().astype(np.uint8)

            if np.sum(ground_truth_mask) == 0 and np.sum(pred_mask_bin) == 0:
                continue

            accuracy = calculate_accuracy(ground_truth_mask, pred_mask_bin)
            dice = dice_coefficient(ground_truth_mask, pred_mask_bin)
            iou = binary_iou(ground_truth_mask, pred_mask_bin)
            
            total_accuracy += accuracy
            total_dice += dice
            total_iou += iou
            num_images_processed += 1

            if iou > highest_iou:
                highest_iou = iou
            if iou < lowest_iou:
                lowest_iou = iou
                lowest_iou_img_path = save_path
                lowest_iou_gt_path = os.path.join(lowest_iou_dir, f'{os.path.splitext(original_filename)[0]}_gt.png')

                gt_mask_img = Image.fromarray(ground_truth_mask * 255)
                gt_mask_img.save(lowest_iou_gt_path)

    if num_images_processed > 0:
        avg_accuracy = total_accuracy / num_images_processed
        avg_dice = total_dice / num_images_processed
        avg_iou = total_iou / num_images_processed
    else:
        avg_accuracy = avg_dice = avg_iou = 0.0

    print(f'Average Accuracy: {avg_accuracy:.4f}')
    print(f'Average Dice Coefficient: {avg_dice:.4f}')
    print(f'Average IoU: {avg_iou:.4f}')
    print(f'Highest IoU: {highest_iou:.4f}')
    print(f'Lowest IoU: {lowest_iou:.4f}')

In [110]:
img_gen, msk_gen, filenames = create_segmentation_generator_test(img_path, BATCH_SIZE)

Found 100 images belonging to 1 classes.
Found 100 images belonging to 1 classes.


In [ ]:
predict_and_save_masks_2(model, img_gen, msk_gen, filenames, output_dir, lowest_iou_dir)

## Overlaying the Predictions onto the Images

In [384]:
import cv2
import numpy as np
import os
import tensorflow as tf

def predict_and_save_masks_2(model, datagen, filenames, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    num_images = len(filenames)
    num_batches = num_images // datagen.batch_size
    remainder = num_images % datagen.batch_size

    def overlay_mask_on_image(image, mask, color=(0, 0, 255), alpha=0.2):
        mask = (mask.squeeze() > 0.5).astype(np.uint8)

        colored_mask = np.zeros_like(image)
        colored_mask[mask == 1] = color

        overlayed_image = cv2.addWeighted(colored_mask, alpha, image, 1 - alpha, 0)

        return overlayed_image

    for i in range(num_batches):
        img_batch = next(datagen)
        
        pred_masks = model.predict(img_batch, verbose=0)
        
        for j, (img, pred_mask) in enumerate(zip(img_batch, pred_masks)):
            original_filename = os.path.basename(filenames[i * datagen.batch_size + j])
            save_path = os.path.join(output_dir, f'{os.path.splitext(original_filename)[0]}_overlay.png')
            
            img = (img * 255).astype(np.uint8)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            
            overlayed_img = overlay_mask_on_image(img, pred_mask)
            cv2.imwrite(save_path, overlayed_img)

    if remainder > 0:
        img_batch = next(datagen)
        pred_masks = model.predict(img_batch[:remainder], verbose=0)
        
        for j, (img, pred_mask) in enumerate(zip(img_batch[:remainder], pred_masks)):
            original_filename = os.path.basename(filenames[num_batches * datagen.batch_size + j])
            save_path = os.path.join(output_dir, f'{os.path.splitext(original_filename)[0]}_overlay.png')
            
            img = (img * 255).astype(np.uint8)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            
            overlayed_img = overlay_mask_on_image(img, pred_mask)
            cv2.imwrite(save_path, overlayed_img)


In [392]:
model = tf.keras.models.load_model("weight_file.h5")
img_path = 'image_path'
output_dir = 'output_path'

In [394]:
gen, filenames = create_segmentation_generator_test(img_path, BATCH_SIZE)

Found 92 images belonging to 1 classes.


In [398]:
predict_and_save_masks_2(model, gen, filenames, output_dir)